### Cái đặt thư viện

In [1]:
%%capture
# We're installing the latest Torch, Triton, OpenAI's Triton kernels, Transformers and Unsloth!
!pip install --upgrade -qqq uv
try: import numpy; install_numpy = f'numpy=={numpy.__version__}'
except: install_numpy = numpy
!uv pip install -qqq \
    torch>=2.8.0 triton>=3.4.0 {install_numpy} \
    unsloth_zoo[base]@git+https://github.com/unslothai/unsloth-zoo \
    unsloth[base]@git+https://github.com/unslothai/unsloth \
    torchvision bitsandbytes \
    git+https://github.com/huggingface/transformers \
    git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels


In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024
dtype = None
model_name = "unsloth/gpt-oss-20b"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    dtype = dtype,  # None for auto detection
    max_seq_length = max_seq_length,  # Choose any for long context!
    load_in_4bit = True,  # 4-bit quantization to reduce memory
    full_finetuning = False,  # [NEW!] We have full finetuning now!
    # token = "hf_xxx",  # nếu là repo private/gated thì cần token HF
)


==((====))==  Unsloth 2025.8.7: Fast Gpt_Oss patching. Transformers: 4.56.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.
Unsloth: Gpt_Oss does not support SDPA - switching to eager!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Add LORA

In [14]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Unsloth: Making `model.base_model.model.base_model.model.model` require gradients


### Thử nghiệm model before

In [16]:
messages = [
    {'role': 'system', 'content': 'Tôi là 1 người dân Việt Nam và đang đi làm thủ tục hành chính ở ủy ban nhân dân', 'thinking': None},
    {"role": "user", "content": "Bạn thường tìm hiều thủ tục hành chính mới bằng cách nào?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low",
).to(model.device)
from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens = 512, streamer = TextStreamer(tokenizer))


<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-19

Reasoning: low

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

Tôi là 1 người dân Việt Nam và đang đi làm thủ tục hành chính ở ủy ban nhân dân<|end|><|start|>user<|message|>Bạn thường tìm hiều thủ tục hành chính mới bằng cách nào?<|end|><|start|>assistant<|channel|>analysis<|message|>Need to answer in Vietnamese, explain methods.<|end|><|start|>assistant<|channel|>final<|message|>Bạn có thể tìm hiểu thủ tục hành chính mới bằng một số cách sau:

| Cách | Mô tả | Lợi ích |
|------|--------|----------|
| **Xem thông tin trên cơ quan chủ quản** | Truy cập trang web chính thức của Ban Chấp hành thủ phủ, các cơ quan hoặc thành phố mà bạn đang làm việc. | Giúp cập nhật chính xác, tránh lỗi chính q

### Load data để fine-tune

In [17]:
def formatting_prompts_func(examples):
    convos = examples['messages']
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {'text': texts}

from datasets import load_dataset
dataset = load_dataset('json', data_files=r"/content/drive/MyDrive/Colab Notebooks/data_tthc/qa_conversations.jsonl", split='train')
dataset


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 1000
})

In [18]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:
print(dataset[0]['text'])

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-19

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>user<|message|>Tôi là 1 người dân Việt Nam và đang đi làm thủ tục hành chính ở ủy ban nhân dân <|end|><|start|>user<|message|>Cá nhân đăng ký Bồi dưỡng nghiệp vụ đăng kiểm viên tàu cá phải nộp những loại giấy tờ gì?<|end|><|start|>assistant<|message|>Theo quy định tại Khoản 3, Điều 8 Thông tư số 23/2018/TT-BNNPTNT ngày 15/11/2018 của Bộ Nông nghiệp và Phát triển nông thôn thì cá nhân đăng ký bồi dưỡng nghiệp vụ đăng kiểm viên tàu cá phải nộp các giấy tờ sau:1) Đơn đề nghị tham gia bồi dưỡng nghiệp vụ đăng kiểm viên tàu cá theo Mẫu số 01.ĐKV Phụ lục II ban hành kèm theo Thông tư này;2) Bản sao văn bằng, chứng chỉ chuyên môn và xuất trình bản chính để đối chiếu đ

### Finetune

In [23]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,  # Set this for 1 full training run.
        # max_steps = 68,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = 'adamw_8bit',
        weight_decay = 0.01,
        lr_scheduler_type = 'linear',
        seed = 3407,
        output_dir = 'outputs',
        report_to = 'none',  # Use this for WandB etc
    ),
)


Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [24]:
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 199998, 'pad_token_id': 200017}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 3,981,312 of 20,918,738,496 (0.02% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,12.049600
2,16.399500
3,16.069400
4,12.368100
5,7.598000
6,5.140200
7,4.337900
8,3.761000
9,3.443800
10,3.112200


### Thử lại sau khi finetune

In [25]:
messages = [
    {'role': 'system', 'content': 'Tôi là 1 người dân Việt Nam và đang đi làm thủ tục hành chính ở ủy ban nhân dân', 'thinking': None},
    {"role": "user", "content": "Bạn thường tìm hiều thủ tục hành chính mới bằng cách nào?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low",
).to(model.device)
from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens = 512, streamer = TextStreamer(tokenizer))


<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-19

Reasoning: low

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

Tôi là 1 người dân Việt Nam và đang đi làm thủ tục hành chính ở ủy ban nhân dân<|end|><|start|>user<|message|>Bạn thường tìm hiều thủ tục hành chính mới bằng cách nào?<|end|><|start|>assistant<|message|>Tại cơ quan làm thủ tục, thường có đăng bản thảo của quyết định hoặc ban hành các số hiệu tàu, tàu phù cắn, tàu phư cảnh, tàu bờ làn không có số làn, tàu lượn không phải lông lông lợn nỉ tàu lưỡng lự tàu lưỡng lũu tàu lưỡng lự tàu lưỡng lũu tàu lưỡng lú; tàu lưỡng lư, tàu lưỡng lướn tàu lưỡng lươn tàu lưỡng lương tàu lưi lư, tàu lưa lư; tàu lưu quay, quay quay, quay quay quay quay quay quay quay quay đi, quay quay quay quay quay 

In [26]:
model.save_pretrained("finetuned_model")

In [27]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = 'finetuned_model',  # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 1024,
        dtype = None,
        load_in_4bit = True,
    )

messages = [
    {'role': 'system', 'content': 'Tôi là 1 người dân Việt Nam và đang đi làm thủ tục hành chính ở ủy ban nhân dân', 'thinking': None},
    {'role': 'user', 'content': 'Thủ tục đăng ký kết hôn như thế nào?'},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = 'pt',
    return_dict = True,
    reasoning_effort = 'low',
).to(model.device)

from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens=512, streamer=TextStreamer(tokenizer))


<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-19

Reasoning: low

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

Tôi là 1 người dân Việt Nam và đang đi làm thủ tục hành chính ở ủy ban nhân dân<|end|><|start|>user<|message|>Thủ tục đăng ký kết hôn như thế nào?<|end|><|start|>assistant<|message|>Phụ kiện của công ước đăng ký kết hôn chưa được quy định về số và tên những hành chính của Bộ trưởng Bộ Phát triển nhân số; Thái Lộ, Bộ Thương Lược; Trường Đại học Quốc quốc và các tổ chức giáo dục, thành lập công ty hoạt động, tổ chức, cơ quan hoạt động; Thái Lộ cá là thủ tục phải có thông báo tại thành tẩm hoặc bở thành tẩm; Thái lộ cá đã thẩm đo quy định này; Thái lộ cá chưa thẩm đo quy định về thái lỗ cá, lạc bướ, cọ, cá tê thị, cá, rắn, châu lục